In [ ]:
import { display } from "tslab";
import { readFileSync } from "fs";

const css = readFileSync("../style.css", "utf8");
display.html(`<style>${css}</style>`);

In [ ]:
import { RecursiveSet, RecursiveMap, Tuple, Value } from "recursive-set";
import { DFA, NFA, State, Char, DFAState } from "./01-NFA-2-DFA";

### DFA Text Representation

The function `dfa2string` converts a deterministic finite automaton (DFA) into a human-readable text format.

**Key Logic:**
* **State Renaming:** Since DFA states can be complex objects (like `RecursiveSet<number>`), we map them to simple names $S_0, S_1, \dots, S_n$ for clarity.
* **State Encoding:** We output a legend mapping each $S_i$ back to its original set value (using `pySetStr`).
* **Transitions:** We list all transitions $\delta(S_i, c) = S_j$.

**Implementation Note:**
We use `state.toString()` as the key for our mapping to ensure robust lookups, avoiding reference identity issues common with object-based sets.

In [ ]:
function dfa2string(dfa: DFA): string {
    const { Q, Σ, δ, q0, A } = dfa;
    let result = "";

    const stateToName = new RecursiveMap<DFAState, string>();
    
    let n = 0;
    for (const q of Q) {
        stateToName.set(q, `S${n++}`);
    }

    result += `states: {S0, ..., S${n - 1}}\n\n`;

    const startName = stateToName.get(q0) ?? "UNKNOWN";
    result += `start state: ${startName}\n\n`;

    result += "state encoding:\n";
    for (const q of Q) {
        result += `${stateToName.get(q)} = ${q.toString()}\n`;
    }

    result += "\ntransitions:\n";
    for (const q of Q) {
        const sourceName = stateToName.get(q)!;

        for (const c of Σ) {
            // SAUBER: Tuple als Key
            const target = δ.get(new Tuple(q, c));

            if (target) {
                const targetName = stateToName.get(target);
                if (targetName) {
                    result += `δ(${sourceName}, ${c}) = ${targetName}\n`;
                }
            }
        }
    }

    result += "\nset of accepting states: {";
    const acceptingNames: string[] = [];

    for (const q of Q) {
        if (A.has(q)) {
            acceptingNames.push(stateToName.get(q)!);
        }
    }
    result += acceptingNames.join(", ");
    result += "}\n";

    return result;
}

### DFA Visualization (DOT Format)

The function `dfa2dot` generates a Graphviz DOT definition for the DFA.

**Features:**
* **Layout:** Uses `rankdir=LR` for a Left-to-Right flow.
* **Start State:** Marked by an invisible "ghost node" pointing to the actual start state.
* **Accepting States:** Drawn with a double periphery (`peripheries="2"`).
* **Return Value:** Returns both the DOT string and the `statesToNames` map, allowing subsequent code to reference the generated state names.

In [ ]:
function dfa2dot(dfa: DFA): { dot: string; statesToNames: RecursiveMap<DFAState, string> } {
    const { Q, Σ, δ, q0, A } = dfa;
    const lines: string[] = [];

    lines.push('digraph "Deterministic FSM" {');
    lines.push("  rankdir=LR;");

    const stateToName = new RecursiveMap<DFAState, string>();
    let n = 0;
    for (const q of Q) {
        stateToName.set(q, `S${n++}`);
    }

    const startName = stateToName.get(q0) ?? "ERR";

    lines.push('  "start_ghost" [label="", width="0.1", height="0.1", style="filled", color="blue"];');
    lines.push(`  "start_ghost" -> "${startName}";`);

    for (const q of Q) {
        const name = stateToName.get(q)!;
        if (A.has(q)) {
            lines.push(`  "${name}" [peripheries="2"];`);
        } else {
            lines.push(`  "${name}";`);
        }
    }

    for (const q of Q) {
        const sourceName = stateToName.get(q)!;

        for (const c of Σ) {
            const target = δ.get(new Tuple(q, c));

            if (target) {
                const targetName = stateToName.get(target);
                if (targetName) {
                    lines.push(`  "${sourceName}" -> "${targetName}" [label="${c}"];`);
                }
            }
        }
    }

    lines.push("}");
    return { dot: lines.join("\n"), statesToNames: stateToName };
}

### NFA Text Representation

The function `nfa2string` converts a non-deterministic finite automaton (NFA) to text.

**Differences from DFA:**
* **Multiple Targets:** A transition `[q, c]` can map to a *set* of target states. We iterate through all targets.
* **Epsilon Transitions:** We specifically check for transitions labeled with $\varepsilon$ (epsilon) or empty string and list them as `[q, ""] |-> p`.

In [ ]:
function nfa2string(nfa: NFA): string {
    const { Q, Σ, δ, q0, A } = nfa;
    let result = "";

    result += `states: ${Q.toString()}\n\n`;
    result += `start state: ${q0}\n\n`;
    result += "transitions:\n";

    for (const q of Q) {
        for (const c of Σ) {
            const targets = δ.get(new Tuple(q, c));
            if (targets && !targets.isEmpty()) {
                for (const p of targets) {
                    result += `[${q}, ${c}] |-> ${p}\n`;
                }
            }
        }

        const targetsEps = δ.get(new Tuple(q, "ε"));
        if (targetsEps && !targetsEps.isEmpty()) {
            for (const p of targetsEps) {
                result += `[${q}, ""] |-> ${p}\n`;
            }
        }
    }

    result += `\nset of accepting states: ${A.toString()}\n`;
    return result;
}

### NFA Visualization (DOT Format)

The function `nfa2dot` visualizes an NFA.

**Visual Cues:**
* **Epsilon Edges:** Transitions labeled $\varepsilon$ are drawn with a lower edge weight (`weight="0.1"`) to encourage Graphviz to place them distinctly, often hinting at the "instant" nature of the transition.
* **Direct Mapping:** Since NFA states are typically simple primitives (numbers/strings), we use their string representation directly as node names (unlike the DFA remapping).

In [ ]:
function nfa2dot(nfa: NFA): string {
    const { Q, Σ, δ, q0, A } = nfa;
    const lines: string[] = [];

    lines.push('digraph "Non-Deterministic FSM" {');
    lines.push("  rankdir=LR;");

    lines.push('  "start_ghost" [label="", width="0.1", height="0.1", style="filled", color="blue"];');
    lines.push(`  "start_ghost" -> "${q0}";`);

    for (const q of Q) {
        const label = q.toString();
        if (A.has(q)) {
            lines.push(`  "${label}" [peripheries="2"];`);
        } else {
            lines.push(`  "${label}";`);
        }
    }

    for (const q of Q) {
        const targetsEps = δ.get(new Tuple(q, "ε"));
        if (targetsEps) {
            for (const p of targetsEps) {
                lines.push(`  "${q}" -> "${p}" [label="ε", weight="0.1"];`);
            }
        }

        for (const c of Σ) {
            const targets = δ.get(new Tuple(q, c));
            if (targets) {
                for (const p of targets) {
                    lines.push(`  "${q}" -> "${p}" [label="${c}", weight="10"];`);
                }
            }
        }
    }

    lines.push("}");
    return lines.join("\n");
}